# X-ray photoelectron spectroscopy (XPS)

## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.ase.espresso.xps import xps_workgraph
from aiida import load_profile
load_profile()

node = xps_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.xps import xps_workgraph
from aiida import load_profile
load_profile()

wg = xps_workgraph()
wg.to_html()

## Example: ETFA molecule

### Prepare the inputs and submit the workflow


In [10]:
from ase.io import read
from aiida import load_profile
from workgraph_collections.ase.espresso.xps import xps_workgraph

load_profile()

# create input structure node
atoms = read("../datas/ETFA.xyz")
atoms.center(vacuum=5.0)
input_data = {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 50,
            "ecutrho": 400,
            "occupations": "fixed",
        },
    }
kpts = (1, 1, 1)
# Pseudos from https://github.com/superstar54/xps-data/tree/main/pseudo_demo/pseudo_demo_pbe
pseudopotentials = {
    "C": "C.pbe-n-kjgipaw_psl.1.0.0.UPF",
    "H": "H.pbe-kjpaw_psl.1.0.0.UPF",
    "O": "O.pbe-n-kjpaw_psl.0.1.UPF",
    "F": "F.pbe-n-rrkjus_psl.1.0.0.UPF",
}
core_hole_pseudos = {
    "C_1s": "C.star1s.pbe-n-kjgipaw_psl.1.0.0.UPF",
    "O_1s": "O.star1s.pbe-n-kjpaw_psl.0.1.UPF",
}
#
metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
scf_inputs = {
    "command": "mpirun -np 4 pw.x",
    "computer": "localhost",
    "metadata": metadata,
    "input_data": input_data,
    "kpts": kpts,
    "pseudopotentials": pseudopotentials,
    "pseudo_dir": "/home/wang_x3/datas/pseudos/xps/pbe",
    "core_hole_pseudos": core_hole_pseudos,
    "is_molecule": True,
}
# corrections from https://github.com/superstar54/xps-data/blob/main/pseudo_demo/pseudo_demo_pbe/datas.py
corrections = {
        "C": 345.99 - 6.2,
        "O": 676.47 - 8.25,
        }
# ===============================================================================
wg = xps_workgraph(
    atoms=atoms,
    atoms_list=[(0, "1s"), (1, "1s"), (2, "1s"), (3, "1s")],
    scf_inputs=scf_inputs,
    corrections=corrections,
    metadata=metadata,
)
wg.name = "ASE-espresso-XPS-ETFA"
wg.submit(wait=True, timeout=300)

WorkGraph node created, PK: 55672


<WorkChainNode: uuid: f0d39e37-6cf4-4724-93cc-822225b2c338 (pk: 55672) (aiida_workgraph.engine.workgraph.WorkGraphEngine)>

## Print the results


In [9]:
for key, energy in wg.nodes["binding_energy"].outputs["result"].value.value.items():
    print(f"{key}: {energy:.2f} eV")

C_1s_0: 298.31 eV
C_1s_1: 295.26 eV
C_1s_2: 293.00 eV
C_1s_3: 291.44 eV


### Run on HPC
Here is an example of how to run the pw.x calculation on the CSCS eiger cluster.


```python 
# run on CSCS eiger cluster
metadata_eiger = {
    "options": {
        'prepend_text' : """
module load cray/22.05  cpeIntel/22.05 QuantumESPRESSO/7.0
eval "$(/users/xingwang/miniconda3/bin/conda shell.posix hook)"
conda activate py3.11
export OMP_NUM_THREADS=1
        """,
        'custom_scheduler_commands' : '#SBATCH --account=mr32',
        'resources': {
                    'num_machines' : 1,
                    'num_mpiprocs_per_machine' : 128,
        }
    }
}
scf_inputs = {
    "command": "srun -n 128 pw.x",
    "computer": "eiger",
    "metadata": metadata_eiger,
    "input_data": input_data,
    "kpts": kpts,
    "pseudopotentials": pseudopotentials,
    # "pseudo_dir": "/home/wang_x3/datas/pseudos/xps/pbe",
    "pseudo_dir": "/users/xingwang/datas/pseudos/xps/pbe",
    "core_hole_pseudos": core_hole_pseudos,
    "is_molecule": True,
}
```